<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 3.0**


---

# Predicting Health Insurance costs

When acquiring Health Insurance, it is common that the we pay a fixed and low amount of money, in return to being covered by the insurance over a high amount of charges during a moment of healthcare need or emergency. Given this fact, is important for insurance companies to predict the cost of customers in case such an event arises, so that their business is still feasible. This is a difficult issue, because it is hard to predict when and how someone will become ill. However, certain aspects of people's behaviour, habits and medical history might be able to tell us how much these patients will cost for the insurance company.

<p align=center>
<img src="img/health_insurance.png" width="30%"><br>
<i><sup>Image credits: pch.vector (<a href="https://br.freepik.com/vetores-gratis/pai-apertando-as-maos-com-agente-de-seguros_6974887.htm">www.freepik.com</a>)</sup></i>
</p>

In this notebook we will be looking at a Health Insurance Cost dataset, using regression machine learning models in [PyCaret](https://pycaret.org/). PyCaret is a popular, low-code library, that provides an automated way to create data analysis workflows using Machine Learning. It aims to reduce time used for coding the models, while leaving more time for the analyses themselves.

# The Data

The data for this project was obtained on [Kaggle](https://www.kaggle.com/annetxu/health-insurance-cost-prediction). There is not much information about it on the page, but it is a simple dataset (with 7 columns, only) which features characteristics of the individuals and their insurance charges over the period analysed (unknown). For ease of access I have downloaded the dataset and included it in the `data` folder for this project.


In [45]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv

# Importing pycaret tools
from pycaret.regression import setup, compare_models, models, create_model, predict_model
from pycaret.regression import tune_model, plot_model, evaluate_model, finalize_model
from pycaret.regression import save_model, load_model

# Getting the data
df = pd.read_csv("data/insurance.csv")

# Life, the Universe, and Everything
np.random.seed(42)

# Defining plot parameters
# plt.style.use('dark_background')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.stretch'] = 'normal'
plt.rcParams['font.style'] = 'normal'
plt.rcParams['font.variant'] = 'normal'

# Checking first entries of the dataset
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [46]:
# Dataset size
df.shape

(1338, 7)

## Data variables

As mentioned above, the dataset comes with 1338 observations and 7 columns only, which are:

* `age` = The age of the individual insurance client.
* `sex` = The biological sex.
* `bmi` = Body Mass Index, a health measure based on weight divided by the squared height.
* `children` = The number of children the individual has.
* `smoker` = If they smoke or not.
* `region` = The region where they live (related to the dataset origin, other information unknown).
* `charges` = The incurred charges originanting from the specific individual. *This is our target variable*.

First, we will observe our variables and make sure that their types match the expected.

In [47]:
# Checking types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [48]:
# Describing our dataset
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


Now, let us use SweetViz for a more compreensive view of our variables.

In [49]:
sweetviz = sv.analyze(df)
sweetviz.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


We can see that our `age` variable only ranges between 18 and 64. If it were there were older people in the dataset, it would be a good idea to create and additional column categorizing the ages, since we know that older people will often need more health care than younger people. Also, the age of the individuals has a somewhat uniform distribution, with the exception of an increase around the lower end of the age range.

The `sex` variable, which might as well influence our predictions is well balanced. The `region` variable is also well balanced.

The `children` variable is more skewed towards the lower end of the distribution, but this is a potential factor that might be associated to our outcome, due to children being able to get ill easier through, for example, school contact.

Our target variable, `charges`, presents with some obvious outliers, but these are of extreme interest in the case of predicting costs in a health insurance scenario.

The `smoker` variable is also unbalanced, with ~20% of smokers in the dataset. This also represents a health risk factor, and the variable will be left as is.

Another possible risk factor for health conditions is **adiposity**. Weight and height alone (the measurements used to calculate BMI - Body Mass Index) are not good enough factors to evaluate someone's adiposity levels and we should be careful not to fat shame other people in the name of their health condition (viewers can read more on it [here](https://www.goodhousekeeping.com/health/a35422452/fat-phobia/)). However, historically the BMI and the categories defined by it have been associate to negative health conditions. Thus, here we will classify the data according to the [BMI standards](https://www.who.int/europe/news-room/fact-sheets/item/a-healthy-lifestyle---who-recommendations) and we will compare how this measurement influences our predictions.

In [50]:
# Copying dataset
df_bmi = df.copy()

# Removing column from other df
df = df.drop('bmi', axis = 1)

In [51]:
# Adding new column with BMI category
# conditions = [
#     (df_bmi['bmi'] < 18.5),
#     (df_bmi['bmi'] >= 18.5) & (df_bmi['bmi'] < 25),
#     (df_bmi['bmi'] >= 25) & (df_bmi['bmi'] < 30),
#     (df_bmi['bmi'] >= 30) & (df_bmi['bmi'] < 35),
#     (df_bmi['bmi'] >= 35) & (df_bmi['bmi'] < 40),
#     (df_bmi['bmi'] >= 40)]
# classes = ['Underweight', 'Regular', 'Overweight', 'Obesity 1', 'Obesity 2', 'Obsesity 3']
# df_bmi['bmi_class'] = np.select(conditions, classes)

df_bmi

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


Now let's separate our train and test datasets:

In [52]:
# Creating test dataset
test = df.sample(frac=0.1)

# Creating train data by dropping test data
train = df.drop(test.index)

# Resetting indexes
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [53]:
# Checking sizes
print(train.shape)
print(test.shape)

(1204, 6)
(134, 6)


In [54]:
# Creating test dataset with bmi
test_bmi = df_bmi.sample(frac=0.1)

# Creating train data by dropping test data
train_bmi = df_bmi.drop(test.index)

# Resetting indexes
train_bmi.reset_index(inplace=True, drop=True)
test_bmi.reset_index(inplace=True, drop=True)

In [55]:
# Checking sizes
print(train_bmi.shape)
print(test_bmi.shape)

(1204, 7)
(134, 7)


Now, let's start with PyCaret.

# Regression with PyCaret

First, we pass our data to PyCaret.

## Creating regressor

In [56]:
# Creating regressor using PyCaret
reg = setup(data=train, target='charges')

,Description,Value
0,session_id,6974
1,Target,charges
2,Original Data,"(1204, 6)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(842, 13)"


From this initial report, we can see that our dataset has no missing values. Thus, we can proceed to create our pipeline.

## PyCaret pipeline

### No BMI information

In [57]:
# Creating pipeline
reg = setup(data=train,
            target='charges',
            normalize=True,
            normalize_method='zscore',
            experiment_name='HealthInsuranceCosts',
            )

,Description,Value
0,session_id,4047
1,Target,charges
2,Original Data,"(1204, 6)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(842, 13)"


Now that our setup has been initiated, let us compare how each regressor model behaves with our dataset.

In [58]:
# Checking regression models
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,4145.3175,40410441.2000,6333.2623,0.6837,0.4521,0.3499,0.0070
llar,Lasso Least Angle Regression,4129.6508,40372145.6117,6330.6693,0.6837,0.4502,0.3482,0.0040
br,Bayesian Ridge,4141.8708,40414730.7437,6333.6311,0.6836,0.4519,0.3489,0.0050
lr,Linear Regression,4124.4184,40415734.2000,6333.8567,0.6833,0.4507,0.3440,0.7230
lasso,Lasso Regression,4123.9067,40413227.4000,6333.6672,0.6833,0.4505,0.3441,0.3040
lar,Least Angle Regression,4243.0670,41958862.5396,6443.3553,0.6713,0.4848,0.3615,0.0040
gbr,Gradient Boosting Regressor,4176.5441,42896320.3111,6526.5508,0.6659,0.4877,0.3858,0.0130
lightgbm,Light Gradient Boosting Machine,4395.2548,45846305.4151,6753.5893,0.6422,0.5608,0.4310,0.0140
catboost,CatBoost Regressor,4284.3970,48228486.7643,6918.5193,0.6279,0.5341,0.4090,0.1530
rf,Random Forest Regressor,4291.6144,49138876.4993,6989.5555,0.6152,0.5015,0.3826,0.0490


### With BMI information

In [59]:
# Creating pipeline
reg = setup(data=train_bmi,
            target='charges',
            normalize=True,
            normalize_method='zscore',
            experiment_name='HealthInsuranceCosts',
            )

,Description,Value
0,session_id,8547
1,Target,charges
2,Original Data,"(1204, 7)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(842, 14)"


In [60]:
# Checking regression models
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2450.9299,20395624.8297,4487.7064,0.8461,0.4332,0.2945,0.0130
rf,Random Forest Regressor,2610.3154,23294721.4055,4794.2119,0.8247,0.4736,0.3281,0.0540
catboost,CatBoost Regressor,2646.8012,23454368.4584,4817.5492,0.8222,0.4960,0.3245,0.2480
lightgbm,Light Gradient Boosting Machine,2864.7157,24330300.3048,4899.1095,0.8161,0.5468,0.3869,0.0160
et,Extra Trees Regressor,2629.5631,25730792.0674,5035.1540,0.8058,0.5016,0.3349,0.0400
ada,AdaBoost Regressor,4676.3021,31403911.1043,5590.5211,0.7591,0.7053,0.8994,0.0070
ridge,Ridge Regression,4152.2051,36184791.4000,5982.1855,0.7264,0.6051,0.4448,0.0040
llar,Lasso Least Angle Regression,4139.7310,36195069.8232,5982.7957,0.7263,0.5961,0.4428,0.0040
br,Bayesian Ridge,4149.8252,36189623.2349,5982.5322,0.7263,0.5958,0.4442,0.0070
lasso,Lasso Regression,4142.8330,36197042.4000,5982.9772,0.7262,0.5901,0.4425,0.0030


From the comparisons, the Gradient Boosting Regressor on the dataset with the BMI information achieved the best metrics in nearly all categories, indicating that this variable still provides us with some prediction power. Let's see how the parameters were set in this model:

In [61]:
# Printing parameters for the best model
print(best)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=8547, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


With our best model identified, we now must effectively build the model to train our data (this is step is not done during `compare_models()`). For comparison purposes, we will also use the second and third best algorithms identified. Since LightGBM is not indicated for datasets with less than 10,000 observations, we will compare it CatBoost and Random Forest Regressor instead.

## The Gradient Boosting Regressor

The Gradient Boosting Regressor, or simply GBR, is a powerful machine learning model used for predictions. It is based on the construction of weak learners, which are improved by adding them together in an ensembl of predictors that minimizes the loss function<sup><a href="https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/">1</a>,<a href="https://en.wikipedia.org/wiki/Gradient_boosting">2</a></sup>.

It basically has three components<sup><a href="https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/">1</a></sup>:

1. A loss function;
2. Weak learners;
3. The additive model in which the weak learners are added to minimize the loss function.

In this model, the weak learners are added one by one, while existing ones remain unchanged. This is done through a [gradient descent](https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html) procedure (iterative method to minimize some function).

### Creating our model

In [62]:
# Creating first model
gbr = create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2440.2893,17970977.7120,4239.2190,0.8526,0.4176,0.3384
1,2597.9785,23128174.8604,4809.1761,0.8199,0.4838,0.3247
2,2326.4187,15751203.7725,3968.7786,0.8482,0.3701,0.2726
3,2613.4333,20903594.8008,4572.0449,0.8717,0.4358,0.3064
4,2953.1232,32656535.7717,5714.5897,0.7874,0.4779,0.2866
5,2162.3931,16585905.7208,4072.5797,0.8448,0.4125,0.2953
6,2546.4933,18101717.6800,4254.6113,0.8824,0.4860,0.4060
7,2194.4696,18449171.8719,4295.2499,0.8238,0.3763,0.2565
8,1994.8695,16457810.8025,4056.8227,0.8938,0.4349,0.2215


This shows the result we had previously, as the model has been instantiated using the same hyperparameters.

### Tuning the GBR model

In [63]:
# Creating tuned model
tuned_gbr = tune_model(gbr, optimize='R2', choose_better=True, n_iter=100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2454.9987,17275883.8708,4156.4268,0.8583,0.4110,0.3628
1,2474.3468,20100174.8106,4483.3218,0.8435,0.4828,0.3926
2,2386.9860,15899356.1459,3987.3997,0.8468,0.4118,0.3201
3,2669.7386,19719238.0179,4440.6349,0.8790,0.4338,0.3369
4,2818.5672,30741092.6068,5544.4650,0.7998,0.4710,0.3199
5,2468.5555,16861202.2912,4106.2394,0.8422,0.5084,0.4529
6,2715.6320,19246801.1762,4387.1176,0.8749,0.5418,0.4973
7,2538.9967,20443403.4228,4521.4382,0.8048,0.4435,0.3603
8,2178.1052,18139783.6774,4259.0825,0.8830,0.4329,0.3133


## Trying CatBoost and Random Forest Regressors

In [64]:
## CatBoost

# Creating model
cat = create_model('catboost')

# Tuning model
tuned_cat = tune_model(cat, optimize='R2', choose_better=True, n_iter=100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2410.8644,16996996.4612,4122.7414,0.8606,0.3841,0.3241
1,2362.1456,20656670.2910,4544.9610,0.8391,0.4543,0.3372
2,2410.7791,17624061.4046,4198.1021,0.8302,0.3959,0.2865
3,2578.1155,20196277.4099,4494.0269,0.8761,0.4301,0.3118
4,2845.8670,31091252.8124,5575.9531,0.7976,0.4626,0.2965
5,2314.1558,15955424.8087,3994.4242,0.8507,0.4661,0.3862
6,2526.3340,18750605.5694,4330.1969,0.8782,0.5262,0.4441
7,2347.8703,19632048.2571,4430.8067,0.8125,0.4103,0.2922
8,2072.1326,17155197.7104,4141.8834,0.8893,0.4122,0.2566


In [65]:
## Random Forest

# Creating model
rf = create_model('rf')

# Tuning model
tuned_rf = tune_model(rf, optimize='R2', choose_better=True, n_iter=100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1816.6516,16408305.0149,4050.7166,0.8654,0.3035,0.1837
1,1850.4625,21665013.2302,4654.5691,0.8313,0.4671,0.1838
2,1955.3282,18836655.2314,4340.1216,0.8185,0.3519,0.1493
3,2094.0844,20559288.2553,4534.2351,0.8738,0.3949,0.1695
4,2291.1112,33335276.2416,5773.6710,0.7829,0.4787,0.1348
5,1547.0883,15964247.0039,3995.5284,0.8506,0.3763,0.1522
6,1853.5845,16839466.5883,4103.5919,0.8906,0.4323,0.1947
7,1852.8936,19344193.4556,4398.2034,0.8153,0.3362,0.1529
8,1578.0637,17388194.4809,4169.9154,0.8878,0.3684,0.1175


# References

1: https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/

2: https://en.wikipedia.org/wiki/Gradient_boosting